<a href="https://colab.research.google.com/github/SaiMithunPunna/AIDS/blob/main/Telco_Churn_Prediction_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.ensemble import BaggingClassifier , AdaBoostClassifier , GradientBoostingClassifier ,RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [48]:
df=pd.read_csv('/content/drive/MyDrive/Colab Datasets/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head()


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [49]:
print(df.isnull().sum());


#display the stats
print(df.describe());


customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64
       SeniorCitizen       tenure  MonthlyCharges
count    7043.000000  7043.000000     7043.000000
mean        0.162147    32.371149       64.761692
std         0.368612    24.559481       30.090047
min         0.000000     0.000000       18.250000
25%         0.000000     9.000000       35.500000
50%         0.000000    29.000000       70.350000
75%         0.000000    55.000000       89.850000
max         1.000000    72.000000      118.750000


In [50]:
df=df.dropna()

In [51]:
label_encoders={}
for column in df.select_dtypes(include=['object']).columns:
  le=LabelEncoder()
  df[column]=le.fit_transform(df[column])
  label_encoders[column]=le

In [52]:
df.head()


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,5375,0,0,1,0,1,0,1,0,0,...,0,0,0,0,0,1,2,29.85,2505,0
1,3962,1,0,0,0,34,1,0,0,2,...,2,0,0,0,1,0,3,56.95,1466,0
2,2564,1,0,0,0,2,1,0,0,2,...,0,0,0,0,0,1,3,53.85,157,1
3,5535,1,0,0,0,45,0,1,0,2,...,2,2,0,0,1,0,0,42.30,1400,0
4,6511,0,0,0,0,2,1,0,1,0,...,0,0,0,0,0,1,2,70.70,925,1


In [53]:
x=df.drop('Churn',axis=1)
y=df['Churn']
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,5375,0,0,1,0,1,0,1,0,0,...,0,0,0,0,0,1,2,29.85,2505,0
1,3962,1,0,0,0,34,1,0,0,2,...,2,0,0,0,1,0,3,56.95,1466,0
2,2564,1,0,0,0,2,1,0,0,2,...,0,0,0,0,0,1,3,53.85,157,1
3,5535,1,0,0,0,45,0,1,0,2,...,2,2,0,0,1,0,0,42.30,1400,0
4,6511,0,0,0,0,2,1,0,1,0,...,0,0,0,0,0,1,2,70.70,925,1


In [54]:
x_train , x_test , y_train , y_test =train_test_split(x , y , test_size=0.2 , random_state=42)
scaler=StandardScaler()
x_train_scaled=scaler.fit_transform(x_train)
x_test_scaled=scaler.transform(x_test)

In [55]:

#bagging

bagging=BaggingClassifier(RandomForestClassifier() , n_estimators=50 , random_state=42)
bagging.fit(x_train_scaled , y_train)
bagging_acc=accuracy_score(y_test , bagging.predict(x_test_scaled))


In [56]:
#ADA BOOST

ada=AdaBoostClassifier(n_estimators=50 , random_state=42)
ada.fit(x_train_scaled , y_train)
ada_acc=accuracy_score(y_test , ada.predict(x_test_scaled))

In [57]:
#Gradient Boosting

gb=GradientBoostingClassifier(n_estimators=100 , learning_rate=0.1 , random_state=42)
gb.fit(x_train_scaled , y_train)
gb_acc=accuracy_score(y_test , gb.predict(x_test_scaled))

In [58]:
#XGB Boosting
xgb=XGBClassifier(n_estimators=100 , learning_rate=0.1 , random_state=42)
xgb.fit(x_train_scaled , y_train)
xgb_acc=accuracy_score(y_test , xgb.predict(x_test_scaled))

In [59]:
import pandas as pd
results_df=pd.DataFrame({
    'Model':['Bagging' , 'AdaBoost' , 'Gradient Boosting' , 'XGBoost'] ,
    'Accuracy':[bagging_acc , ada_acc ,gb_acc , xgb_acc ]
})
results_df

,Model,Accuracy
0,Bagging,0.804826
1,AdaBoost,0.807665
2,Gradient Boosting,0.809084
3,XGBoost,0.805536
